In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import re
from dateutil.parser import parse
from collections import defaultdict
from datetime import datetime
from ast import literal_eval

In [13]:
# Creating auxiliary function
def export_json(data, filename):    
    with open(f'./output/{filename}.json', 'w') as f:
        json.dump(data, f)

# General cleaning

In [14]:
# Importing the data
df_books = pd.read_csv('best_books_goodreads.csv', dtype={'awards':object})

# Dropping unnecessary columns
df_books.drop(columns=['description', 'language', 'characters', 'bookFormat','edition','publisher','likedPercent',
                        'setting', 'coverImg', 'bbeScore', 'bbeVotes', 'price'], inplace=True)

print('Columns of dataset')
print(df_books.columns)

# Removing books without ISBN
print('-> Lenght before removing books without ISBN:',len(df_books))
df_books = df_books[df_books['isbn']!= '9999999999999']
print('-> Lenght after removing books without ISBN:',len(df_books))

# Removing books with low quantity of ratings
median_num_ratings = df_books['numRatings'].fillna(value=0).astype(int).quantile(0.5)
print("-> Lenght before filtering 0.5 best books:", len(df_books))
df_books = df_books[df_books['numRatings'] > median_num_ratings].copy()
print("-> Lenght after filtering 0.5 best books:", len(df_books))

# Removing books with weird or uncommon page counts
print("-> Lenght before filtering by weird number of pages:", len(df_books))
df_books = df_books[df_books['pages']!='1 page'] # '1 page' error
df_books = df_books[~df_books['pages'].isna()] # NA error
df_books = df_books[df_books['pages'] != 0] # Books with 0 pages
min_num_of_pages = 10
max_num_of_pages = 2000 
df_books['pages'] = df_books['pages'].astype(int)
df_books = df_books[df_books['pages'] < max_num_of_pages] # Removing books with less than 10 pages
df_books = df_books[df_books['pages'] > min_num_of_pages] # Removing books with more than 2000 pages
print("-> Lenght after filtering by weird number of pages:",len(df_books))

Columns of dataset
Index(['bookId', 'title', 'series', 'author', 'rating', 'isbn', 'genres',
       'pages', 'publishDate', 'firstPublishDate', 'awards', 'numRatings',
       'ratingsByStars'],
      dtype='object')
-> Lenght before removing books without ISBN: 52478
-> Lenght after removing books without ISBN: 48124
-> Lenght before filtering 0.5 best books: 48124
-> Lenght after filtering 0.5 best books: 24060
-> Lenght before filtering by weird number of pages: 24060
-> Lenght after filtering by weird number of pages: 23711


### Cleaning date column

In [15]:
df_books[['publishDate', 'firstPublishDate']]

,publishDate,firstPublishDate
0,09/14/08,NaN
1,09/28/04,06/21/03
4,09/06/06,10/05/05
5,03/14/06,09/01/05
6,04/28/96,08/17/45
...,...,...
52453,October 1st 1991,1991
52461,September 19th 2014,NaN
52463,October 1st 2015,NaN
52469,January 1st 1984,May 1st 1976


In [16]:
# Function to clean date
def clean_date(date_str):
    try:
        # If the date contains only a year, returns None
        if re.match(r'^\d{4}$', date_str):
            return None
        
        # If the date is in MM/DD/YY format
        if re.match(r'\d{2}/\d{2}/\d{2}', date_str):
            parsed_date = pd.to_datetime(date_str, format='%m/%d/%y')
        else:
            # If the date is in a text format, use the date parser
            parsed_date = parse(date_str)
        
        # If the result date is after than 2020, correct it to 19XX
        if parsed_date >= datetime.strptime('2021-01-01', '%Y-%m-%d'):
            parsed_date = parsed_date.replace(year=parsed_date.year - 100)
        
        # Returns date in YYYY-MM-DD format
        return parsed_date.strftime('%Y-%m-%d')
    except:
        return None
    
df_books['best_publish_date'] = df_books['firstPublishDate'].fillna(df_books['publishDate'])

# Apply the function to the 'dirty_date' column
df_books['clean_date'] = df_books['best_publish_date'].apply(clean_date)

# Removes books with no publishing date
print("-> Lenght before filtering null or weird dates:",len(df_books))
df_books = df_books[~df_books['clean_date'].isna()]

# Removes books with weird date
df_books = df_books[df_books['clean_date']!='0458-09-24']
print("-> Lenght after filtering null or weird dates:",len(df_books))

# Calculates day of the week and month based on publishing date
df_books['clean_date'] = pd.to_datetime(df_books['clean_date'])
df_books['day_of_week'] = df_books['clean_date'].dt.day_name().astype(str)
df_books['month_name'] = df_books['clean_date'].dt.month_name().astype(str)

-> Lenght before filtering null or weird dates: 23711
-> Lenght after filtering null or weird dates: 22972


### Cleaning genres

In [17]:
# Formatting the genres column
df_books['genres'] = df_books['genres'].apply(literal_eval)

# Filtering the genre with a hand tailored list
valid_genres = ['Fiction', 'Nonfiction', 'Mystery', 'Fantasy', 'Science Fiction', 'Romance', 'Biography', 'Historical Fiction', 'Young Adult', 'Childrens', 'Self Help', 'Horror', 'Classics', 'Poetry', 'Graphic Novels', 'Adventure', 'True Crime', 'Religion', 'Science', 'Business']
df_books['valid_genres'] = df_books['genres'].apply(lambda x: [item for item in x if item in valid_genres])


# Calculating success metrics

In [18]:
# Creating normalized rating column
df_books['norm_rating'] = df_books['rating']/5

# Calculating number of awards per book
df_books['awards'] = df_books['awards'].apply(literal_eval)
df_books['num_awards'] = df_books['awards'].apply(len)

# Creating normalized number of awards column
max_num_awards = max(df_books['num_awards'])
df_books['norm_num_awards'] = df_books['num_awards']/max_num_awards

# Creating normalized number of ratings column
max_num_ratings = max(df_books['numRatings'])
df_books['norm_num_ratings'] = df_books['numRatings']/max_num_ratings

# Defining the weights for a general success metric
weights = {
    'avg_rating': 1,
    'num_awards': 2,
    'num_ratings': 1
}

# Calculating the success rate
df_books['success_rate'] = (df_books['norm_num_awards']*weights['num_awards']+\
                           df_books['norm_num_ratings']*weights['num_ratings']+\
                           df_books['norm_rating']*weights['avg_rating'])/sum(weights.values())

# Creating auxiliary objects

### Success metrics by number of pages

In [26]:
# Calculating the average success rate by number of pages
success_by_num_pages = df_books.groupby('pages').agg({
    'success_rate':'mean',
    'norm_rating':'mean',
    'norm_num_awards':'mean',
    'norm_num_ratings':'mean',
    })

success_by_num_pages.to_csv('output/num_pages_analysis/sucess_by_num_of_pages.csv')

### Number of books by time

In [34]:
# Calculating and exporting number of books by date
num_book_by_date = df_books.groupby('clean_date').agg({'bookId': 'count'})
num_book_by_date.rename(columns={'bookId': 'num_of_books'}, inplace=True)
num_book_by_date.to_csv('./output/num_books_by_time/num_book_by_date.csv')

# Calculating and exporting number of books by day of week
num_book_by_day_of_week = df_books.groupby('day_of_week').agg({'bookId': 'count'})
num_book_by_day_of_week.rename(columns={'bookId': 'num_of_books'}, inplace=True)
num_book_by_day_of_week.to_csv('./output/num_books_by_time/num_book_by_day_of_week.csv')

# Calculating and exporting number of books by month name
num_book_by_month = df_books.groupby('month_name').agg({'bookId': 'count'})
num_book_by_month.rename(columns={'bookId': 'num_of_books'}, inplace=True)
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
num_book_by_month = num_book_by_month.reindex(new_order, axis=0)
num_book_by_month.to_csv('./output/num_books_by_time/num_book_by_month.csv')

### Books and book count by genre

In [35]:
# Getting the books for each genre
genre_dict = defaultdict(list)
for index, row in df_books.iterrows():
    book_name = row['bookId']
    genres = row['valid_genres']
    for genre in genres:
        genre_dict[genre].append(book_name)

books_by_genre_dict = dict(genre_dict)

# Get the number of books for each genre and sort it
genre_count = {k: len(v) for k, v in books_by_genre_dict.items()}
genre_count = {k: v for k, v in sorted(genre_count.items(), key=lambda item: item[1], reverse=True)}

# Saving number of books for each genre and books by genre
export_json(genre_count, 'genres/num_books_by_genre')
export_json(books_by_genre_dict, 'genres/books_by_genre')

### Creating object with genres co-occurences

In [36]:
# Initialize nested defaultdict for counting co-occurrences
co_occurrence = defaultdict(lambda: defaultdict(int))

# Count co-occurrences for each genre combination
for genre1 in books_by_genre_dict:
    for genre2 in books_by_genre_dict:
        if genre1 == genre2:
            continue
        common_books = set(books_by_genre_dict[genre1]) & set(books_by_genre_dict[genre2])
        co_occurrence[genre1][genre2] = len(common_books)

# Convert nested defaultdict to regular dict
co_occurrence = {k: dict(v) for k, v in co_occurrence.items()}

# Saving co-occurence counts
export_json(co_occurrence, 'genres/cooccurences_2_by_2_genre')

# Getting trends in series

### Creating dataframe with books that are series only

In [37]:
# Creating columns with the name and position in series
df_series = (df_books[~df_books['series'].isna()]).copy()
df_series['name_of_series'] = df_series['series'].str.split(pat='#', expand=True)[0]
df_series['num_in_series'] = df_series['series'].str.split(pat='#', expand=True)[1]
# df_series.head(3)

# Filtering for coherent series numbers (from 1 to 20)
valid_series = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
df_series = df_series[df_series['num_in_series'].isin(valid_series)].copy()
df_series['num_in_series'] = df_series['num_in_series'].astype(int)

### Creating auxiliary objects for the series dataset

In [39]:
# Calculating and exporting success rates by number in series
sucess_by_num_in_series = df_series.groupby('num_in_series', as_index=True).agg({
    'success_rate':'mean',
    'norm_rating':'mean',
    'norm_num_awards':'mean',
    'norm_num_ratings':'mean',
})
sucess_by_num_in_series.to_csv('./output/series_analysis/sucess_by_num_in_series.csv', index=True)

# Exporting final dataframes

In [40]:
df_books.to_csv('output/output_dfs/clean_books.csv', index=False)
df_series.to_csv('output/output_dfs/series_books.csv', index=False)